In [1]:
from basic_nlp import *
from json_io import *
from ml import *

C:\Program Files\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
def keyToStr(d):
    new = {}
    for key,value in d.items():
        new[str(key)] = value
    return new

def unigramFeature(tweet):
    tokens = tokenize(tweet)
    tagged = pos(tokens)
    chunked = chunk(tagged)
    tokens = tokNoNE(chunked)
    ull = upperLowerLen(tokens)
    cases = wordCases(ull)
    tagged = pos(tokens)
    lemma  = lemmatize(tagged)
    puncuationFreqDict = punctuationFeatures(tweet)
    for key,val in puncuationFreqDict.items():
        puncuationFreqDict[key]=val[2]
    
    capFreq = capLetterFreq(ull)
    allCapsFreq = cases.count('AC')/len(cases)
    unigrams = keyToStr(dict(freq(grams(tokens, 1))))
    bigrams = keyToStr(dict(freq(grams(tokens, 2))))
    trigrams = keyToStr(dict(freq(grams(tokens, 3))))
    feat = {}
    feat.update(unigrams)
    feat.update(bigrams)
    feat.update(trigrams)
    feat.update(puncuationFreqDict)
    feat.update({"capFreq":capFreq, "allCapsFreq":allCapsFreq})
    return feat

In [3]:
sarcasticTweets = tweet_iterate("../json/sarcastic/unique.json", key="text")
sarcasticFeats = [(unigramFeature(repr(next(sarcasticTweets))), True) for x in range(0,100)]

seriousTweets = tweet_iterate("../json/non_sarcastic/unique.json", key="text")
seriousFeats = [(unigramFeature(repr(next(seriousTweets))), False) for x in range(0,100)]

TypeError: punctuationFeatures() missing 1 required positional argument: 's'

In [ ]:
from random import shuffle
featTup = sarcasticFeats + seriousFeats
shuffle(featTup)
(feats, bools) = list(zip(*featTup))
vec = createFeatVector(feats)[1]

In [ ]:
from sklearn.naive_bayes import GaussianNB
m = ml({"naive bayes":GaussianNB()})

In [ ]:
m.trainVectorizedFeatures(vec[:50], bools[:50])

In [ ]:
m.accuracyVectorizedFeatures(vec[50:], bools[50:])